In [1]:
''' модуль загрузки библиотек и преподготовки базы '''
import re
from dotenv import load_dotenv
import openai
import re
import time
import json
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import importlib
import mdTOdocx
import requests
import os

''' модуль загрузки данных из файла и очистки от личной информации '''
# функця считывания ключа openAI из .env
def get_key_ОpenAI():
    # подгружаем переменные окружения
    load_dotenv()
    # токен бота
    openai.api_key = os.environ.get("OPENAI_API_KEY")

def remove_personal_data(text):
    # Удаление email адресов
    text = re.sub(r'\S+@\S+', '<email>', text)
    # Удаление имен и фамилий (примерный паттерн)
    text = re.sub(r'\b[A-Z][a-z]+ [A-Z][a-z]+\b', '<name>', text)
    # Удаление номеров телефонов
    text = re.sub(r'\+?\d[\d -]{8,}\d', '<phone>', text)
    # Удаление номеров билетов и бронирования
    text = re.sub(r'\b\d{6,10}\b', '<ticket_number>', text)
    # Удаление номеров паспортов
    text = re.sub(r'\b[A-Z0-9]{8,10}\b', '<passport_number>', text)
    return text

def make_dirs():
      """ модуль создания необходимых каталогов """
      base_dir = 'base/'
      # проверяем наличие каталога 'base' при необходимости создаем
      if not os.path.exists(base_dir):
            # Если папка не существует, создаем её
                  os.makedirs(base_dir)

      # проверяем наличие подкаталогов 'input','output','structured'. При необходимости создаем
      base_dirs = ['input','output','structured', 'promts'] # папки которые должны быть созданы внутри каталога base
      for d in base_dirs:
            if not os.path.exists(f'{base_dir}{d}'):
            # Если папка не существует, создаем её
                  os.makedirs(f'{base_dir}{d}')


In [2]:
''' модуль функций. обработки писем - извлечение вопрос/ответ и категоризация '''
def call_gpt_api(prompt):
    ''' функция обращения к chatGPT'''

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            return response.choices[0].message['content']
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
            time.sleep(60)
        except openai.error.OpenAIError as e:
            print(f"An error occurred: {e}")
            return None

def extract_segments(cleaned_text):
    ''' функция извлечения писем из обработанного файла txt '''

    segments = cleaned_text.split("## Текст письма №")
    return segments

def process_segment(segment):
    ''' функция первой обработки письма - извлечение вопрос/ответ '''

    prompt = f"""
    Act as a specialist in extracting entities from text and preserving confidential information. 
    Your task is to extract the question and answer from the document, and then check the question and answer for confidential information (surname, name, patronymic, passport number, ticket number, reservation number, phone number). If such information is present, delete it. If the question and answer do not contain useful information for passenger support service - delete these messages. Summarize the question and answer, leaving only the essence. Do not make up anything on your own. Base your question only on the information provided to you. 
    If there is no question or answer, delete these messages. 
    If you need to provide support contacts in your reply, please specify e-mail: MEGACOMPANY@MEGACOMPANY.com tel: +0-00-000-000.
    Write in Russian! If the question or answer is not written in Russian, please translate it into Russian!
        
    Message: {segment}
    
    JSON output format:
    {{
        "question": "",
        "answer": ""
    }}
    """
    result = call_gpt_api(prompt)
    return result

def extract_qa_pairs(segments):
    ''' функция первой обработки всех писем - извлечение вопрос/ответ '''

    aq_dict = []
    for segment in segments:
        processed = process_segment(segment)
        
        # print(f'\nОТрабатываем сегмент:\n{segment}\n')
        # print(f'Вывод функции process_segment\n{processed}\n')
        
        if processed != '{question:, answer:}':
            try:
                processed_dict = json.loads(processed) 
                if processed_dict['question'] and processed_dict['answer']:

                    # print(f'Добавляем в словарь:\n{processed_dict}')

                    aq_dict.append(processed_dict)
                
            except:
                print(f'\nисключен  результат process_segment\n{processed}\n')
    return aq_dict

def remove_duplicate_questions_list(qa_pairs):
    ''' функция удаления дубликатов из пар вопрос/ответ'''

    questions = [pair['question'] for pair in qa_pairs]
    vectorizer = TfidfVectorizer().fit_transform(questions)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    to_remove = set()
    for i in range(len(cosine_matrix)):
        for j in range(i+1, len(cosine_matrix)):
            if cosine_matrix[i][j] > 0.85:  # Порог схожести
                to_remove.add(j)
    
    unique_qa_pairs = [qa_pairs[i] for i in range(len(qa_pairs)) if i not in to_remove]
    return unique_qa_pairs

def determine_categories(questions:list):
    ''' функция определния категорий из конечного списка вопросов '''

    all_questions = ", ".join(questions)
    # all_questions=questions
    prompt = f"Answer only in Russian. Based on the following questions, identify no more than 20 main categories and 20 subcategories:\n{all_questions}."
    categories = call_gpt_api(prompt)
    return categories

def classify_question(question, categories):
    ''' функция определения категории одного вопроса '''

    prompt = f"""
    К какой из представленных  категорий относится следующий вопрос? 
    \nВопрос:{question}\nКатегории:{categories}.
    Выведи ответ в формате json:
    {{"category": ""}}
    """
    category = call_gpt_api(prompt)
    if not category:
        print("Received empty response from API.")
        return None

    try:
        # Очистим строку от лишних символов
        clean_category = category.strip().replace('```json', '').replace('```', '')

        # Используем регулярное выражение для извлечения текста между фигурными скобками
        match = re.search(r'(\{.*?\})', clean_category)

        if match:
            clean_category = match.group(1)
            category_json = json.loads(clean_category)
            return category_json
        else:
            # print("Категории не определны в нужном формате")
            category_json = json.loads(clean_category)
            return category_json
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        print(f"Response was: {category}")
        return None

def create_knowledge_base(qa_pairs_json:json, categories):
    ''' функция создания базы знаний из окончательно обработанных пар вопрос/ответ '''

    sections = {category: [] for category in categories.split("\n") if category}
    sections["Прочие"] = []  # Обязательно добавляем категорию "Прочие"

    # Получение всех подкатегорий
    subcategories = "\n".join([category for category in sections.keys() if category and category.strip().startswith('-')])

    # print(subcategories) #!!!!!!!!!!!

    for pair in json.loads(qa_pairs_json):
        question, answer = pair['question'], pair['answer']
        category = classify_question(question, subcategories)
        

        # print(f'\nДля вопроса \n{question}\nопределена категория:{category}\n')

        added = False  # Флаг для отслеживания, была ли категория добавлена
        if category and 'category' in category:
            for category_key in sections.keys():
                if category['category'] in category_key:
                    sections[category_key].append(pair)
                    added = True
                    break

        if not added:  # Если ни одна категория не подошла, добавляем в "Другое"
            sections["Прочие"].append(pair)
            
    knowledge_base = ""
    for section, pairs in sections.items():

        # print(f'\nsection:\n {section}\n')

        # корректно разобьем маркдуаун  на категории и подкатегории 
        if section.strip().startswith('-'):          
            knowledge_base += f"## {section}\n"
        else:
            knowledge_base += f"# {section}\n"

        for pair in pairs:
            knowledge_base += f"### Вопрос: {pair['question']}\n- Ответ: {pair['answer']}\n"

    return knowledge_base

def markdown_to_dataframe(md_file_path):
    ''' функция конвертации md файла базы знаний в df pandas '''

    with open(md_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    data = {'Category': [], 'Subcategory': [], 'Question': [], 'Answer': []}
    current_category = None
    current_subcategory = None
    
    for i, line in enumerate(lines):
        if line.startswith('# '):
            current_category = line[2:].strip()
            current_subcategory = None
        elif line.startswith('## '):
            current_subcategory = line[3:].strip()
        elif line.startswith('### Вопрос: '):
            # question = line[10:].strip().rstrip('**')
            question = line[11:].strip().rstrip('**')
            answer_index = i + 1
            if answer_index < len(lines) and lines[answer_index].startswith('- Ответ:'):
                answer = lines[answer_index][9:].strip()
                data['Category'].append(current_category)
                data['Subcategory'].append(current_subcategory)
                data['Question'].append(question)
                data['Answer'].append(answer)
    
    df = pd.DataFrame(data)
    return df


def dataframe_to_markdown(df, md_file_path):
    ''' функция обратной конвертации файла базы знаний из df pandas в md  '''

    with open(md_file_path, 'w', encoding='utf-8') as file:
        for category in df['Category'].unique():
            file.write(f'# {category}\n')
            category_df = df[df['Category'] == category]
            for subcategory in category_df['Subcategory'].unique():
                if subcategory and subcategory != "None":
                    file.write(f'## {subcategory}\n')
                else:
                    pass
                    # file.write(f'## Без подкатегории\n')  # Добавлено для обработки None
                subcategory_df = category_df[category_df['Subcategory'] == subcategory]
                for _, row in subcategory_df.iterrows():
                    question = row['Question']
                    answer = row['Answer']
                    file.write(f"### Вопрос: {question}\n- Ответ: {answer}\n")

def remove_duplicate_questions(df):
    ''' Повторная проверка и удаление одинаковых по смыслу вопросов (из df pandas) '''

    vectorizer = TfidfVectorizer().fit_transform(df['Question'])
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    to_remove = []
    for i in range(len(cosine_matrix)):
        for j in range(i+1, len(cosine_matrix)):
            if cosine_matrix[i][j] > 0.5:  # Порог схожести
                to_remove.append(j)
    
    df = df.drop(to_remove)
    return df

def clean_with_chatgpt(qa_pairs):
    ''' вторая обработка базы знаний через chatGPT '''

    cleaned_pairs = []
    for pair in qa_pairs:
        question = pair.get("question", "").strip()
        answer = pair.get("answer", "").strip()

        # Prompt for ChatGPT to clean the data
        prompt = f"""
        Вопрос: {question}
        Ответ: {answer}
        ---
        Удали все персональные данные и удали пары, где ответ не является корректным или не содержит полезной информации. Переформулируй ответы и вопросы, если это необходимо.
        Вот так должны выглядеть идеальные пары вопрос-ответ:
        
        Где я могу посмотреть самые дешевые варианты перелета?
        При поиске минимальные тарифы будут первыми в предложенном списке.

        Как получить маршрут-квитанцию на удобном мне языке?
        Маршрут-квитанция предоставляется на трех языках (казахском, русском и английском).

        Как правильно указать фамилию и имя латиницей при покупке билета?
        Правильная транслитерация фамилии и имени указана в вашем паспорте/удостоверении личности.

        Как я могу оформить билет для младенца до 2-х лет?
        Если билет был куплен на официальном сайте авиакомпании www.MEGACOMPANY.com, вам необходимо отправить запрос в Авиакомпанию с документами младенца, прикрепив билет взрослого. Если же билет был приобретен на стороннем сайте или кассе, необходимо обратиться по месту приобретения билета.
        """

        cleaned_text = call_gpt_api(prompt)

        # cleaned_text = response.choices[0].text.strip()
        if cleaned_text:
            cleaned_pair = cleaned_text.split('\n')
            if len(cleaned_pair) == 2:
                cleaned_question = cleaned_pair[0].replace("Вопрос: ", "").strip()
                cleaned_answer = cleaned_pair[1].replace("Ответ: ", "").strip()
                if cleaned_question and cleaned_answer:
                    cleaned_pairs.append({
                        "question": cleaned_question,
                        "answer": cleaned_answer
                    })
    return cleaned_pairs

# Function to clean question-answer pairs using ChatGPT
def clean_with_chatgpt_else(qa_pairs):
    ''' третья обработка базы знаний через chatGPT '''

    cleaned_pairs = []
    for pair in qa_pairs:
        question = pair.get("question", "").strip()
        answer = pair.get("answer", "").strip()

        # Prompt for ChatGPT to clean the data
        prompt = f"""
        Преобразуй вопросы пассажиров и ответы службы клиентской поддержки авиакомпании MEGACOMPANY, которые я напишу  следующим образом:
        - измени ответ так, чтобы он содержал в начале саму суть вопроса, в виде оборота для ответа 
        Пример: 
        **Вопрос: как мне выполнить большую задачу?**
        Правильный Ответ: Чтобы выполнить большую задачу необходимо разбить ее на подзадачи и решить их отдельно.
        Неправильный Ответ: Необходимо разбить ее на подзадачи и решить их отдельно.

        - если в вопросе есть просьба, то преобразуй ее в вопрос (Например "Помогите сделать" -> "Как мне сделать?")
        - оставь в вопросе и ответе только самое  важное
        - убери из ответов и вопросов детали  конкретного случая (например конкретные детали заказа: города и аэропорты, даты, время вылета, имена пассажиров). Если данная ситуация не может повториться у другого пассажира и вопрос не является типовым  - напиши "Соединяю с оператором"
        - Обобщи ответ или вопрос если упоминается  конкретная организация. Например вместо "банк Сбер" необходимо указать "банк"
        - учти, что все билеты куплены в авиакомпании MEGACOMPANY
        - не выводи контактные данные авиакомпании
        - убери все рекомендации связаться с авиакомпанией

        Если вопрос или ответ бессмысленные, напиши "Игнорирую"

        Пример:
        Вопрос: **Как я могу изменить дату бронирования билетов в Самару для семьи из трех человек? Купил в агентстве Каспи**
        - неправильный Ответ: Для изменения даты бронирования билетов для семьи из трех человек в Самару вам необходимо связаться с агентством Каспи.
        - Правильный Ответ: Для изменения даты бронирования билетов  вам необходимо связаться с агентством, в котором был куплен билет.

        Пример:
        Неправильный Вопрос: **Как исправить имя в билете, где указано неправильное имя ILIA вместо ИЛИЯ?**
        Правильный вопрос: **Как исправить имя в билете?**

        Вопрос: {question}
        Ответ: {answer}
        """

        cleaned_text = call_gpt_api(prompt)

        # cleaned_text = response.choices[0].text.strip()
        if cleaned_text:
            cleaned_pair = cleaned_text.split('\n')
            if len(cleaned_pair) == 2:
                cleaned_question = cleaned_pair[0].replace("Вопрос: ", "").strip()
                cleaned_answer = cleaned_pair[1].replace("Ответ: ", "").strip()
                if cleaned_question and cleaned_answer:
                    cleaned_pairs.append({
                        "question": cleaned_question,
                        "answer": cleaned_answer
                    })
    return cleaned_pairs

def read_json(file_path):
    ''' Функция для чтения JSON из файла '''

    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def write_json(data, file_path):
    ''' Функция для записи JSON в файл '''

    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def load_document_text(url: str, docx_file:str) -> str:
    ''' функция для загрузки документа docx по ссылке из гугл драйв '''

    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # print(doc_id)    
    # print(f'https://docs.google.com/document/d/{doc_id}/export?format=docx')
    
    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=docx')
    response.raise_for_status()
    text = response.content

    with open(docx_file, 'wb') as f:
        f.write(response.content)

    return text

def md_to_excel(md_file_path:str, excel_file:str):
    ''' конвертация md в excel '''
    # md_file_path = f'{base_dir}/output/base_final.md'
    df = markdown_to_dataframe(md_file_path)

    # сохраним в excel для проверки
    df.to_excel(excel_file)   # !!!! раскоментить перед правкой
    return

''' правка excel '''

def excel_to_md(excel_file:str, md_file_path:str):
    ''' конвертация excel в md '''
   
    # загрузим из excel
    df = pd.read_excel(excel_file)  # !!!! раскоментить после правки

    # сохрание базы в md файл
    dataframe_to_markdown(df, md_file_path)
    return

def google_docx_to_md(docx_url:str, docx_file:str):
    ''' функция загрузки базы из google docx, конвертация в md и сохранения на диске '''


    # docx_file = 'base/output/base_final.docx'
    # docx_url = 'https://docs.google.com/document/d/1idE1tls-N.............'
    load_document_text(docx_url, docx_file)

    # конвертация в md для базы знаний
    mdTOdocx.convert_docx_to_md(docx_file, 'base/output/base_final.md')

    return

def md_to_dict_list(md_file_path):
    ''' функция перевода базы из md в список словарей для последущей обработки '''

    # перевод базы в df  и удаление дубликатов
    md_file_path = 'base/output/base_final.md'
    df = markdown_to_dataframe(md_file_path)
    
    def create_json(row):
        return {'question':row['Question'],'answer':row['Answer']}

    clean_unique_qa_pairs = list(df.apply(create_json, axis=1))

    # возращаем список словарей если понадобиться
    return clean_unique_qa_pairs

In [3]:
def first_round_make_base(cleaned_data:str) -> list:
    ''' модуль создания чернового варианта базы знаний из очищенной переписки '''
    # # Загрузка очищенных данных из файла base base/output/emails.txt
    # with open(output_filename, 'r', encoding='utf-8') as file:
    #     cleaned_data = file.read()

    # Извлечение сегментов сообщений 
    segments = extract_segments(cleaned_data)  #  !!!! Ограничиваем первые 10000 символов для тестирования

    # Извлечение вопросов и ответов с помощью chatGPT
    qa_pairs = extract_qa_pairs(segments)

    # Удаление дублирующихся вопросов
    unique_qa_pairs = remove_duplicate_questions_list(qa_pairs)

    # qa_pairs_json = json.dumps(unique_qa_pairs, ensure_ascii=False, indent=4)

    # # выводим вопросы и ответы в разные списки
    # questions = [item["question"] for item in unique_qa_pairs]
    # answers = [item["answer"] for item in unique_qa_pairs]

    # запишем черновой вариант базы в json
    write_json(unique_qa_pairs, 'base/output/unique_qa_pairs.json')
    return unique_qa_pairs

In [4]:
def second_round_make_base(unique_qa_pairs:list) -> list:
    """ модуль повторной проверки и очистки вопрос/ответ  """

    # повторный прогон базы через чат гпт для удаления неккоректных данных
    cleaned_unique_qa_pairs = clean_with_chatgpt(unique_qa_pairs)

    # print(len(cleaned_unique_qa_pairs))

    # Удаление дублирующихся вопросов
    double_clean_unique_qa_pairs = remove_duplicate_questions_list(cleaned_unique_qa_pairs)

    # # перевод в json формат
    # double_clean_unique_qa_pairs_json = json.dumps(double_clean_unique_qa_pairs, ensure_ascii=False, indent=4)

    # # извлечение вопросов и ответов
    # questions = [item["question"] for item in double_clean_unique_qa_pairs]
    # answers = [item["answer"] for item in double_clean_unique_qa_pairs]

    # сохраним в файл json итоговый вариант очищенной базы 
    write_json(double_clean_unique_qa_pairs, 'base/output/double_clean_unique_qa_pairs.json')
    len(double_clean_unique_qa_pairs)
    return double_clean_unique_qa_pairs

In [5]:
def triple_round_make_base(double_clean_unique_qa_pairs:list) -> list:
    """ третий прогон через chatGPT: проверка и очистка вопрос/ответ """
    
    # загрузка из два раза обработанного файла
    double_clean_unique_qa_pairs = read_json('base/output/double_clean_unique_qa_pairs.json')

    # третий раунд очистки через GPT
    triple_clean_unique_qa_pairs = clean_with_chatgpt_else(double_clean_unique_qa_pairs)

    # сохраним резултат в json
    write_json(triple_clean_unique_qa_pairs, 'base/output/triple_clean_unique_qa_pairs.json')
    # print(len(triple_clean_unique_qa_pairs))
    # triple_clean_unique_qa_pairs

    
    return triple_clean_unique_qa_pairs

In [6]:
def categories_make(triple_clean_unique_qa_pairs:list):
    ''' определения перечня категорий из итоговых вариантов вопрос/ответ'''

    # Определение категорий
    questions = [item["question"] for item in triple_clean_unique_qa_pairs]
    categories = determine_categories(questions[:150]) # ограничение по токенам для модели gpt-3.5-turbo-0125 16000 токенов. Поэтому выводим не более 150 вопросов

    # запишем в файл категории
    with open('base/output/categories.txt', 'w', encoding='UTF8') as f:
        f.write(categories)

    # print("Определенные категории и подкатегории:")
    # print(categories)
    return categories

In [7]:
def make_base(triple_clean_unique_qa_pairs, categories):
    ''' модуль создания базы знаний из троекратно обработанного файла почты '''

    # # загрузим категории из файла base/output/categories.txt
    # with open('base/output/categories.txt', 'r', encoding='UTF8') as f:
    #     categories = f.read()
    # print(categories)

    # создадим json для корректной работы create_knowledge_base
    triple_clean_unique_qa_pairs_json = json.dumps(triple_clean_unique_qa_pairs, ensure_ascii=False, indent=4)

    # Создание базы знаний с помощью chatGPT
    knowledge_base = create_knowledge_base(triple_clean_unique_qa_pairs_json, categories)

    # Сохраненим в файл
    with open('base/output/base_final.md', 'w', encoding='utf-8') as file:
        file.write(knowledge_base)

    # сохраним в excel    
    df = markdown_to_dataframe('base/output/base_final.md')
    # # сохраним в excel для проверки
    df.to_excel('base/output/base.xlsx')   

    print("Создание базы знаний завершено.")
    return

In [26]:
''' главный модуль программы '''
# проверим наличие и при необходимости создадим требуемые каталоги
make_dirs()

base_dir = 'base/'                                    # зашито в make_dirs
filename = base_dir + 'input/emails.json'             # путь к обрабатываемому файлу исходящей почтой
output_filename = base_dir + 'output/emails.txt'      # путь к обработанному файлу почты 
structured_file_dir = base_dir + 'structured'         # каталог к временным структурированным файлам обработанными в chatGPT
faiss_db = f'{base_dir}output/faiss_db'               # база FAISS для сохранения
base_file = f'{base_dir}output/base.txt'              # итоговый файл базы размеченный в маркдаун

""" модуль парсинга почты из json"""
# расскоментить если начинаем парсить почту из json
import email_parser
# парсим input/emails.json и сохраняем в output/emails.txt
email_parser.main(filename, output_filename)
""" -------------------------------"""

# Загрузка почты  из файла txt
with open(output_filename, 'r', encoding='utf-8') as file:
    data = file.read()

# Очистка данных
cleaned_data = remove_personal_data(data)

# Сохранение очищенных данных в новый файл
with open(f'{base_dir}/output/cleaned_emails.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_data)

print("Очистка данных завершена.")
print(f"Всего символов в файле:{len(cleaned_data):,}")

""" создание базы """
unique_qa_pairs = first_round_make_base(cleaned_data[:50000])    # производим первый прогон частично обработанных писем через GPT   
print('После первого прогона получили пар вопрос/ответ: ', len(unique_qa_pairs))   
double_clean_unique_qa_pairs = second_round_make_base(unique_qa_pairs) # производим второй прогон через GPT извлеченных пар вопрос/ответ
print('После второго прогона получили пар вопрос/ответ: ', len(double_clean_unique_qa_pairs))
triple_clean_unique_qa_pairs = triple_round_make_base(double_clean_unique_qa_pairs) # производим третий прогон через GPT извлеченных пар вопрос/ответ
print('После третьего прогона получили пар вопрос/ответ: ', len(triple_clean_unique_qa_pairs))
categories = categories_make(triple_clean_unique_qa_pairs)  # определяем категории полученных вопросов
print('получили категорий и подкатегорий: ', len(categories.replace('\n\n','\n').split('\n')))

# загрузим категории из файла base/output/categories.txt. После создания категорий можно проверить и скорректировать вручную
with open('base/output/categories.txt', 'r', encoding='UTF8') as f:
    categories = f.read()
print(categories)

make_base(triple_clean_unique_qa_pairs, categories) # создаем базу. сохраняем сразу в md и excel

# конвертим ее в docx для заливки на google docs
mdTOdocx.convert_md_to_docx('base/output/base_final.md', 'base/output/base_final.docx')

Очистка данных завершена.
Всего символов в файле:1,463,057

исключен  результат process_segment
Спасибо за ваш запрос. Пожалуйста, предоставьте текст с вопросом и ответом для извлечения и обработки.


исключен  результат process_segment
Спасибо за предоставленную информацию. Пожалуйста, дайте мне немного времени, чтобы извлечь и обработать вопрос и ответ из сообщения.

После первого прогона полчили пар вопрос/ответ:  42
После второго прогона полчили пар вопрос/ответ:  18
После третьего прогона полчили пар вопрос/ответ:  4
получили категорий:  482
Создание базы знаний завершено.
